In [137]:
import cv2
import os
import numpy as np
import tensorflow as tf
from random import Random
from tqdm import tqdm
from matplotlib import pyplot as plt

rng = Random(1410)

In [138]:
COCO_IMAGE_PATH = '/home/pwl/Projects/tiny-tracking/COCO/val2017'

In [139]:
target_shapes = {
    'yolox': (416, 416),
    'yolov8': (640, 640),
    'efficientdet': (320, 320),

}

f32_model_paths = {
    'yolox': '/home/pwl/Projects/tiny-tracking/source_models/yolox_nano/yolox_nano_ti_lite_26p1_41p8',
    'yolov8': '/home/pwl/Projects/tiny-tracking/source_models/yolov8/yolov8_with_signatures',
    'efficientdet': '/home/pwl/Projects/tiny-tracking/source_models/efficientdet/efficientdet_lite0_detection_1_with_signatures',
    
}

In [140]:
image_files = [f for f in os.listdir(COCO_IMAGE_PATH)]
rng.shuffle(image_files)
selected_images = image_files[:500]

def representative_dataset_gen():
    for image_name in tqdm(selected_images):
        image_path = os.path.join(COCO_IMAGE_PATH, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        resized_img = cv2.resize(img, target_shapes['efficientdet'])
        resized_img = np.expand_dims(resized_img, axis=0)
        resized_img = resized_img.astype(np.float32)
        yield [resized_img]

In [141]:
converter = tf.lite.TFLiteConverter.from_saved_model(f32_model_paths['efficientdet'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
tflite_quant_model = converter.convert()
open("yolox_nano_dynamic_quant.tflite", "wb").write(tflite_quant_model)

2023-07-18 14:56:44.888961: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-18 14:56:44.888987: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-18 14:56:44.889160: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/pwl/Projects/tiny-tracking/source_models/efficientdet/efficientdet_lite0_detection_1_with_signatures
2023-07-18 14:56:44.936107: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-18 14:56:44.936148: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/pwl/Projects/tiny-tracking/source_models/efficientdet/efficientdet_lite0_detection_1_with_signatures
2023-07-18 14:56:45.066572: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-18 14:56:45.766307: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on Save

3936672